# load packages

In [50]:
import pandas as pd
import numpy as np
import pickle as pkl

# load data

## id_name

In [51]:
apptype_id_name = pd.read_csv('../apptype_id_name.txt',header=None,sep='\\t',encoding='utf8',names=['app_id','app_label'])
apptype_id_name['app_id'] = apptype_id_name['app_id'].astype('str')
apptype_id_name.head()

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,app_id,app_label
0,1401,便捷生活
1,1402,游戏
2,1403,通讯社交
3,1404,阅读
4,1405,工作求职


In [52]:
# apptype_id_name['app_id'].value_counts()
primary_id = apptype_id_name[apptype_id_name['app_id'].apply(lambda x: len(x)<=4)]
secondary_id = apptype_id_name[apptype_id_name['app_id'].apply(lambda x: len(x)>4)]
del apptype_id_name

In [53]:
primary_id = primary_id.rename(columns={'app_id':'primary_id'})
secondary_id['primary_id'] = secondary_id['app_id'].str.slice(0,4)

In [54]:
id_label = primary_id.merge(secondary_id,on='primary_id',how='outer')

In [55]:
id_label = id_label.set_index(['primary_id','app_label_x'])
id_label.head()

app_id app_label_y
primary_id app_label_x                    
1401       便捷生活         140101          打车
           便捷生活         140102        地图导航
           便捷生活         140103      免费WIFI
           便捷生活         140104          租车
           便捷生活         140105          代驾

In [56]:
# id_label['app_label_y'].value_counts(dropna=False)
id_label[id_label['app_label_y']=='音乐']

,,app_id,app_label_y
primary_id,app_label_x,,
1402,游戏,140208,音乐
1406,影音娱乐,140603,音乐


In [57]:
label_num,unique_id = id_label['app_id'].factorize()
id_dict = dict(zip(label_num,unique_id))
id_dict

{0: '140101',
 1: '140102',
 2: '140103',
 3: '140104',
 4: '140105',
 5: '140106',
 6: '140107',
 7: '140108',
 8: '140109',
 9: '140110',
 10: '140111',
 11: '140112',
 12: '140113',
 13: '140114',
 14: '140201',
 15: '140202',
 16: '140203',
 17: '140204',
 18: '140205',
 19: '140206',
 20: '140207',
 21: '140208',
 22: '140209',
 23: '140210',
 24: '140211',
 25: '140212',
 26: '140213',
 27: '140214',
 28: '140301',
 29: '140302',
 30: '140303',
 31: '140304',
 32: '140305',
 33: '140306',
 34: '140307',
 35: '140308',
 36: '140401',
 37: '140402',
 38: '140403',
 39: '140404',
 40: '140405',
 41: '140406',
 42: '140407',
 43: '140408',
 44: '140410',
 45: '140411',
 46: '140412',
 47: '140501',
 48: '140502',
 49: '140601',
 50: '140602',
 51: '140603',
 52: '140604',
 53: '140605',
 54: '140606',
 55: '140607',
 56: '140701',
 57: '140702',
 58: '140703',
 59: '140704',
 60: '140705',
 61: '140706',
 62: '140707',
 63: '140708',
 64: '140709',
 65: '140801',
 66: '140802',
 67: 

## train_data

In [58]:
apptype_train = pd.read_csv('../apptype_train.dat',header=None,sep='\\t',encoding='utf8',names=['app_name','app_id','app_desc'])
apptype_train.head()

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,app_name,app_id,app_desc
0,00000777CE5B5AA5C1AC94DB8EABE0AC,140203,《游戏王座》使用说明书成分由怪兽卡、魔法卡、陷阱卡合计数千张卡牌以及刺激性、耐久性玩法组成。...
1,0000DEC36E15C27DBFC64AB8208C4B37,140206,更稳定、更优质，邀您一起。
2,0001791406307B1D1CE2BC64A830B7C7,142106,《小钱袋》是一款免费网络版记帐软件，适用于个人记帐、家庭记帐、团队记帐，全程帮您安全记录您财...
3,0002F14825B9CA01653325EEFD69D790,142701,领先的周易服务平台高人汇，汇聚算命大师、风水大师、占卜大师、手相大师、起名大师、算命先生、面...
4,000419D79365331F89399E5F38A91B05,140901,平行空间是一款极简、免费的黑科技双开助手；您可以在平行空间双开微信微博、陌陌、映客、yy等应...


## split app_id

In [59]:
app_id = apptype_train['app_id'].str.split('|',expand=True)
app_id.columns = ['app_id','app_id_']
app_id.head()

,app_id,app_id_
0,140203,None
1,140206,None
2,142106,None
3,142701,None
4,140901,None


In [60]:
apptype_train = pd.concat([apptype_train.drop('app_id',axis=1),app_id],axis=1)
apptype_train.head()

,app_name,app_desc,app_id,app_id_
0,00000777CE5B5AA5C1AC94DB8EABE0AC,《游戏王座》使用说明书成分由怪兽卡、魔法卡、陷阱卡合计数千张卡牌以及刺激性、耐久性玩法组成。...,140203,None
1,0000DEC36E15C27DBFC64AB8208C4B37,更稳定、更优质，邀您一起。,140206,None
2,0001791406307B1D1CE2BC64A830B7C7,《小钱袋》是一款免费网络版记帐软件，适用于个人记帐、家庭记帐、团队记帐，全程帮您安全记录您财...,142106,None
3,0002F14825B9CA01653325EEFD69D790,领先的周易服务平台高人汇，汇聚算命大师、风水大师、占卜大师、手相大师、起名大师、算命先生、面...,142701,None
4,000419D79365331F89399E5F38A91B05,平行空间是一款极简、免费的黑科技双开助手；您可以在平行空间双开微信微博、陌陌、映客、yy等应...,140901,None


In [61]:
apptype_train = pd.DataFrame(np.vstack((apptype_train[['app_name','app_desc','app_id']].values,
                                 apptype_train[['app_name','app_desc','app_id_']].dropna().values)),
                                  columns=['app_name','app_desc','app_id'])
apptype_train.head()

,app_name,app_desc,app_id
0,00000777CE5B5AA5C1AC94DB8EABE0AC,《游戏王座》使用说明书成分由怪兽卡、魔法卡、陷阱卡合计数千张卡牌以及刺激性、耐久性玩法组成。...,140203
1,0000DEC36E15C27DBFC64AB8208C4B37,更稳定、更优质，邀您一起。,140206
2,0001791406307B1D1CE2BC64A830B7C7,《小钱袋》是一款免费网络版记帐软件，适用于个人记帐、家庭记帐、团队记帐，全程帮您安全记录您财...,142106
3,0002F14825B9CA01653325EEFD69D790,领先的周易服务平台高人汇，汇聚算命大师、风水大师、占卜大师、手相大师、起名大师、算命先生、面...,142701
4,000419D79365331F89399E5F38A91B05,平行空间是一款极简、免费的黑科技双开助手；您可以在平行空间双开微信微博、陌陌、映客、yy等应...,140901


## drop useless app_desc

In [62]:
apptype_train = apptype_train[apptype_train['app_desc'].apply(lambda x:len(x)>=4)] 

apptype_train = apptype_train[apptype_train['app_desc']!='更稳定、更优质，邀您一起。'] # test中有
apptype_train = apptype_train[apptype_train['app_desc']!='部分小错误，整体。'] # test中有
apptype_train = apptype_train[apptype_train['app_desc']!='了一些BUG。'] # test中有
apptype_train = apptype_train[apptype_train['app_desc']!='该应用内部中，暂不提供下载'] # test中有

apptype_train['app_desc'] = apptype_train['app_desc'].str.replace('了一些BUG。','')
apptype_train['app_desc'] = apptype_train['app_desc'].str.replace('部分小错误，整体。','')
apptype_train['app_desc'] = apptype_train['app_desc'].str.replace('更稳定、更优质，邀您一起。','')
apptype_train['app_desc'] = apptype_train['app_desc'].str.replace('部分。','')
apptype_train['app_desc'] = apptype_train['app_desc'].str.replace('提升体验，修复部分问题。','')
apptype_train['app_desc'] = apptype_train['app_desc'].str.replace('更新内容修复','')
apptype_train['app_desc'] = apptype_train['app_desc'].str.replace('修复','')
apptype_train['app_desc'] = apptype_train['app_desc'].str.replace('BUG','')
apptype_train['app_desc'] = apptype_train['app_desc'].str.replace('和错误','')
apptype_train['app_desc'] = apptype_train['app_desc'].str.replace('优化体验，增强稳定性。','')

In [63]:
app_desc30 = apptype_train[apptype_train['app_desc'].apply(len)<30]
app_desc30.shape

(1670, 3)

In [64]:
drop_index = [72,306,404,571,1271,2153,2568,3132,3186,5141,6182,6197,6329,6492,
                6906,7956,8069,8195,8780,10426,10511,11535,12831,14140,14307,14455,
                14679,15551,16969,17848,17892,18675,19209,19740,20631,22638,23259,
                23695,25316,27006,27516,27584,28360,28488,29967,29982]

# app_desc30[1600:1650]
apptype_train = apptype_train.drop(index=drop_index)
apptype_train.shape

(30875, 3)

## app_id ==> label

In [65]:
dict_id = {}
for k,v in id_dict.items():
    dict_id[v] = k
apptype_train['label'] = apptype_train['app_id'].replace(dict_id)
apptype_train.drop('app_id',axis=1,inplace=True)

In [66]:
# apptype_train[apptype_train['app_desc'].apply(lambda x:len(x)<4)]

## test_data

In [67]:
apptype_test = pd.read_csv('../app_desc.dat',header=None,sep='\\t',encoding='utf8',names=['app_name','app_desc'])
apptype_test.head()

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,app_name,app_desc
0,BB29DA6F8167CFC99E0853741C4EB17B,注意]游戏需要在设备上自己的歌曲注意]音乐赛车是一个音乐改编的赛车游戏，你用你自己的音乐比赛...
1,BB2A78EA7AD4945EAF6E38997F6139A3,定位试衣到家是一款基于地理位置，提供试衣到家专属购物体验的互联网平台。购物流程客户在试衣到家...
2,BB2B1604CFA079C289FECF927DFBCE89,"想念一个人,就说出来。记得要下载安卓锁屏才可正常显示锁屏效果哦~更新内容更稳定、更优质，邀您..."
3,BB2C7BD0B0623644183DAD08A89E1D90,闽通宝手机客户端是基于移动互联网的，以公众出行服务为基础，贯彻绿色出行，低碳生活的理念，为出...
4,BB2E1A8F56158E483D7461E930E6332F,风靡全球的DIY照片桌面，干净、流畅，启动提速100，瞬间提升手机性能；更是一亿用户的共同选...


In [68]:
apptype_test[apptype_test['app_desc'].apply(lambda x:len(x)<5)]

,app_name,app_desc
10660,FD980DC9275D72749015E42DF1D6E531,"""二，"
13393,064BD72AD58055D27DC60A0D1E81B7E4,实时美颜
20860,1A58C51F9FC2354ED9C821DE9879F0F4,一、
27350,2BA90EBBD28A7AE20A7CA17077DCA6E8,详细描述
30971,359312689984EAF2EE54D6FF48B20DF0,指导单位
32835,3AA368B37AFA234F35DA3DDE84E2986D,重要信息
37528,47424E78ACC2B554A8D2FEC3606119B1,应用介绍
40539,4F35F932FD78C2423F049EF53E8140C3,应用介绍
58471,7FFF8A6AA1504DE5F173CB36718CB66D,应用描述
70424,A03FB8D125761066E5CB0CE2C9506F57,一、


## train_data+test_data

In [69]:
train_test_data = pd.concat([apptype_train,apptype_test],sort=False)
train_test_data = train_test_data.reset_index(drop=True)
train_test_data.at[0,'app_desc'] = train_test_data['app_desc'][0][:229]
train_test_data.head()

,app_name,app_desc,label
0,00000777CE5B5AA5C1AC94DB8EABE0AC,《游戏王座》使用说明书成分由怪兽卡、魔法卡、陷阱卡合计数千张卡牌以及刺激性、耐久性玩法组成。...,16.0
1,0001791406307B1D1CE2BC64A830B7C7,《小钱袋》是一款免费网络版记帐软件，适用于个人记帐、家庭记帐、团队记帐，全程帮您安全记录您财...,103.0
2,0002F14825B9CA01653325EEFD69D790,领先的周易服务平台高人汇，汇聚算命大师、风水大师、占卜大师、手相大师、起名大师、算命先生、面...,125.0
3,000419D79365331F89399E5F38A91B05,平行空间是一款极简、免费的黑科技双开助手；您可以在平行空间双开微信微博、陌陌、映客、yy等应...,74.0
4,0004749ACB5303DD2820CD4EE503E44D,和教育全国版是中国移动面向中小学师生和家长推出的一款集在线教育学习及家校互动沟通为一体的应用...,56.0


# opencc
繁体->简体

In [74]:
from opencc import OpenCC
opcc = OpenCC('t2s')

app_desc = []
for desc in train_test_data['app_desc']:
    app_desc.append(opcc.convert(str(desc)))
train_test_data['app_desc'] = app_desc

In [71]:
# train_test_data['app_desc'][130873]

# translate
日文->中文

In [95]:
from google_translate import google_translate

zh_desc = google_translate(train_test_data['app_desc'])
# train_test_data['app_desc'] = zh_desc

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=13052): Max retries exceeded with url: /session/eff6c501a3777234c14cbe2b69b3d08e/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000192CF845898>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。',))

# app_desc->word2vec

## 去掉标点符号

In [36]:
import re
import string
punctuations = string.punctuation + "、。“ ”《 》，：；？! * — （ ）￥·‘’〈〉…＜＞＠＃＄％︿＆※＊＋～｜［］「」｛｝▼◆■☆★"
regex = re.compile('[{}]'.format(re.escape(punctuations)))
train_test_data['app_desc'] = train_test_data['app_desc'].str.replace(regex,' ')

In [37]:
train_test_data.head()

,app_name,app_desc,label
0,00000777CE5B5AA5C1AC94DB8EABE0AC,游戏王座 使用说明书成分由怪兽卡 魔法卡 陷阱卡合计数千张卡牌以及刺激性 耐久性玩法组成 ...,16.0
1,0001791406307B1D1CE2BC64A830B7C7,小钱袋 是一款免费网络版记帐软件 适用于个人记帐 家庭记帐 团队记帐 全程帮您安全记录您财...,103.0
2,0002F14825B9CA01653325EEFD69D790,领先的周易服务平台高人汇 汇聚算命大师 风水大师 占卜大师 手相大师 起名大师 算命先生 面...,125.0
3,000419D79365331F89399E5F38A91B05,平行空间是一款极简 免费的黑科技双开助手 您可以在平行空间双开微信微博 陌陌 映客 yy等应...,74.0
4,0004749ACB5303DD2820CD4EE503E44D,和教育全国版是中国移动面向中小学师生和家长推出的一款集在线教育学习及家校互动沟通为一体的应用...,56.0


## 分词
这里因为要训练词向量，词汇越多越好，分词的时候暂不进行去停

In [38]:
import jieba
import jieba.analyse

In [39]:
with open('./appdict.txt','r',encoding='utf8') as f:
    suggest_words = f.read().split()
    for word in suggest_words:
        jieba.suggest_freq(word,tune=True)

# '/'.join(jieba.cut(train_test_data['app_desc'][1]))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\win10\AppData\Local\Temp\jieba.cache
Loading model cost 0.802 seconds.
Prefix dict has been built succesfully.


In [40]:
cut_app_desc = []
for desc in  train_test_data['app_desc']:
    cut_app_desc.append(' '.join(jieba.cut(desc)).replace(' ,',''))

In [41]:
train_test_data['app_desc'] = cut_app_desc
train_test_data.head()

,app_name,app_desc,label
0,00000777CE5B5AA5C1AC94DB8EABE0AC,游戏王座 使用 说明书 成分 由 怪兽卡 魔法卡 陷阱卡 合计 数千张 卡牌...,16.0
1,0001791406307B1D1CE2BC64A830B7C7,小钱袋 是 一款 免费 网络版 记帐 软件 适用 于 个人 记帐 家庭 记帐...,103.0
2,0002F14825B9CA01653325EEFD69D790,领先 的 周易 服务平台 高人汇 汇聚 算命 大师 风水 大师 占卜 大师 ...,125.0
3,000419D79365331F89399E5F38A91B05,平行 空间 是 一款 极简 免费 的 黑 科技 双开 助手 您 可以 在 平行 空间...,74.0
4,0004749ACB5303DD2820CD4EE503E44D,和 教育 全国 版 是 中国移动 面向 中小学 师生 和 家长 推出 的 一款 集 在线教育...,56.0


In [42]:
train_test_data = train_test_data.dropna(subset=['app_desc'])
train_test_data.to_csv('./train_test_data.csv',index=False) # space separate

## train wv

### Word2Vec

In [119]:
# frogensimsim.models import KeyedVectors
# model = KeyedVectors.load_word2vec_format('F:/corpus/zhwiki-corpus-w2v.vector')
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import multiprocessing

F:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [120]:
with open('./app_desc.txt','w',encoding='utf8') as f:
    for desc in train_test_data['app_desc']:
        f.write(desc+'\n')
# # sg=0:CBOW sg=1:n-grams
model = Word2Vec(LineSentence('./app_desc.txt'),size=400,window=10,min_count=1,sg=1,workers=multiprocessing.cpu_count())

F:\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [121]:
model.save('./app_desc.model')

### FastText

In [1]:
from gensim.models import FastText

F:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
model = FastText.load('F:/corpus/zhwiki-corpus-fasttext.model')

F:\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [38]:
# model.wv.index2word.index('欧几里得')
# model.wv.word_vec('欧几里得')
# model.wv.vectors[0]
train_test_data[train_test_data['app_desc'].apply(lambda x:len(x.split()))>2000]

,app_name,app_desc,label
12643,4F6489DC1A5A42E6A09AC67228C6A6C6,◆ ア ン サ ー な ら 「 话 せ な か っ た こ と 」 「 も っ と 话 し ...,56.0
20010,7D6E38DFCEB4341E1C9BDF033566B584,★ 利用 数 700 万人 突破 ★ ☆ GooglePly 无料 位 获得 ☆ 地震 台风...,74.0
124464,DEEA81F44F0A806674F9B3E2F991FDEC,「 端末 と ア プ リ の 履 歴 」 の 権 限 に つ い て iSPEED ア プ ...,NaN


In [40]:
train_test_data['app_desc'][12643]

'◆ ア ン サ ー な ら 「 话 せ な か っ た こ と 」 「 も っ と 话 し た い こ と 」 全部 话 せ ま す ◆ に も 言 え な い 恋 の 悩 み 相谈 か ら 热中 し て い る 趣味 の 话 ま で カ テ ゴ リ に 分 か れ て い る か ら 幅 広 く な ん で も 话 せ ま す ・ 匿名 だ か ら 安心 し て 本音 で 话 せ ま す ・ 利用 は す べ て 无料 で 累计 ☆ レ ビ ュ ー 000 件 以上 の 満 足度 ・ 写真 投稿 で コ ー デ ィ ネ ー ト ア ド バ イ ス ボ イ ス 机能 で ラ ジ オ 配信 动画 机能 で お 料理 実 况 な ど 色 々 な コ ミ ュ ニ ケ ー シ ョ ン を 楽 し め ま す 気 に な る 话题 を つ け た ら ど ん ど ん 気 軽 に コ メ ン ト し よ う ◆ ア ン サ ー を 使 う と ど う な る の 日常生活 も ち ょ っ と 楽 し く な り ま す ・ 困 っ て い る 人 の 话 を 闻 い て あ げ て 力 に な れ ま す ・ 大势 の 人 に 励 ま し て も ら っ て 勇 気 が で ま す ・ み ん な に は 言 え な い 内绪 の 相谈 を す る と 背中 を 押 し て く れ ま す ・ 友达 が 盛 り が っ て い る 话题 に も つ い て い け る よ う に な り ま す ・ マ ン ネ リ 解消 デ ー ト を 教 わ っ て 彼女 と 円 満 に ご せ ま す ・ 先辈 マ マ か ら の 励 ま し で 出产 前 の 不安 を 解消 で き ま す ◆ こ ん な 人 に オ ス ス メ ・ 大好 き な 漫画 の 话题 で 盛 り が り た い ・ ど う や っ た ら れ た 人 と 复 縁 で き る の ・ 简 単 な お つ ま み レ シ ピ が 知 り た い ・ 话题 の ニ ュ ー ス を 彻底 议论 し た い ・ か の 役 に ち た い ・ と に か く 今 か と 笑 い た い ・ 辛 く て 眠 れ な い か ら か 相手 し て 欲 し い ・ デ ー ト に う コ ー デ ィ ネ ー ト っ て な

# remove stop_words
去停后部分为空

In [9]:
train_test_data = pd.read_csv('./train_test_data.csv')

In [14]:
stop_words = [word.strip('\n') for word in open('./stopwords.txt','r',encoding='utf8')]
desc_remove_stopwords = []
for desc in train_test_data['app_desc']:
    desc = [w for w in desc.split() if w not in stop_words]
    desc_remove_stopwords.append(' '.join(desc))

In [15]:
train_test_data['app_desc'] = desc_remove_stopwords

In [16]:
train_test_data.to_csv('./train_test_data_remove_stopwords.csv',index=False) # str

# extract keywords
提取后更多为空

In [52]:
train_test_data = pd.read_csv('./train_test_data_remove_stopwords.csv')
train_test_data.loc[:,'app_desc'] = train_test_data['app_desc'].fillna('')

In [55]:
# jieba.analyse.set_idf_path()
# jieba.analyse.extract_tags(train_test_data['app_desc'][2], topK=20, withWeight=True)
desc_key_words = []
for desc in train_test_data['app_desc']:
    desc_key_words.append(jieba.analyse.textrank(desc,topK=10))

In [56]:
train_test_data['app_desc'] = list(map(''.join,desc_key_words))

In [57]:
train_test_data.to_csv('./train_test_data_keywords.csv',index=False) #str

# build models

## prepare data

In [2]:
from gensim.models import Word2Vec
wiki_model = Word2Vec.load('../corpus/zhwiki-corpus-w2v.model')
app_model = Word2Vec.load('../corpus/app_desc.model')

In [11]:
train_test_data = pd.read_csv('./train_test_data_remove_stopwords.csv')
# train_test_data = pd.read_csv('./train_test_data.csv')
train_test_data.loc[:,'app_desc'] = train_test_data['app_desc'].fillna('')

In [3]:
train_num = sum(train_test_data['label'].notnull())
train_num

30875

In [7]:
# from sklearn.model_selection import train_test_split
# train_test_data[:train_num].to_csv('./train_data.csv',index=False)
# train_test_data[train_num:].to_csv('./test_data.csv',index=False)

# train_train,train_valid = train_test_split(train_test_data[:train_num])
# train_train.to_csv('./train_train.csv',index=False)
# train_valid.to_csv('./train_valid.csv',index=False)

In [5]:
# app_model_vocab = {}
# for k,v in app_model.wv.vocab.items():
#     if k in app_model_vocab:
#         app_model_vocab[k] += app_model.wv.vocab[k].count
#     else:
#         app_model_vocab[k] = app_model.wv.vocab[k].count


def w2v_features(text=train_test_data['app_desc'],w2v_model=wiki_model,w2v_model_=app_model,res='mean'):
    max_len = text.apply(lambda x: len(x.split())).max()
    app_desc_wv = []
    for desc in text:
        desc_wv = []
        if desc.split()==[]:
            wv = np.zeros(400)
            desc_wv.append(wv)
        else:
            for word in desc.split():
                if word in w2v_model.wv.vocab:
                    wv = w2v_model.wv.get_vector(word)
                else:
                    try:
                        wv = w2v_model_.wv.get_vector(word)
                    except:
                        most_similar_word = w2v_model.most_similar(word)[0][0]
                        wv = w2v_model.wv.get_vector(word)
                desc_wv.append(wv)
                
        if res=='mean':
            desc_wv = np.vstack(desc_wv)
            desc_wv = np.mean(desc_wv,axis=0)
            app_desc_wv.append(desc_wv)
        elif res=='sum':
            desc_wv = np.vstack(desc_wv)
            desc_wv = np.sum(desc_wv,axis=0)
            app_desc_wv.append(desc_wv)
#         elif res=='concat':
#             desc_wv = np.hstack(desc_wv+[np.zeros((max_len-len(desc_wv))*400)])
#             app_desc_wv.append(desc_wv)
            
    app_desc_wv = np.vstack(app_desc_wv)
    return app_desc_wv

In [6]:
w2v_features = w2v_features(res='sum')

In [7]:
from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# BOW = CountVectorizer()
# X = BOW.fit_transform(train_test_data['app_desc'].values[:train_num])
# test_X = BOW.transform(train_test_data['app_desc'].values[train_num:])
# TFIDF = TfidfVectorizer()
# X = TFIDF.fit_transform(train_test_data['app_desc'][:train_num])
# test_X = TFIDF.transform(train_test_data['app_desc'][train_num:])

X = w2v_features[:train_num]
y = train_test_data['label'].values[:train_num]
train_x,test_x,train_y,test_y = train_test_split(X,y)

## logit regression

In [10]:
from sklearn.linear_model import LogisticRegression
# from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score,f1_score
# from scipy.sparse import csr_matrix

In [14]:
# lr_model = LogisticRegression(penalty='l1',C=3,n_jobs=-1,solver='liblinear') #'newton-cg'l2, 'saga' and 'lbfgs'l2
lr_model = LogisticRegression(penalty='l2',C=3,n_jobs=-1,solver='newton-cg')
lr_model.fit(train_x,train_y)

y_pred = lr_model.predict(test_x)

train_score = lr_model.score(train_x,train_y)
test_score = lr_model.score(test_x,test_y)
acc = accuracy_score(test_y,y_pred)
f1 = f1_score(test_y,y_pred,average='micro')

print("train_score:",train_score,"\n","test_score:",test_score)
print('acc:',acc,'\n','f1:',f1)

## SVC

In [14]:
from sklearn.svm import SVC

In [ ]:
svc_model = SVC(C=3,kernel='linear')
svc_model.fit(train_x,train_y)

y_pred = svc_model.predict(test_x)

train_score = svc_model.score(train_x,train_y)
test_score = svc_model.score(test_x,test_y)
acc = accuracy_score(test_y,y_pred)
f1 = f1_score(test_y,y_pred,average='micro')

print("train_score:",train_score,"\n","test_score:",test_score)
print('acc:',acc,'\n','f1:',f1)

In [71]:
with open ('./svc_model.pkl','wb') as f:
    pkl.dump(svc_model,f)

## NB

In [8]:
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB

In [11]:
nb_model = BernoulliNB()
# nb_model = MultinomialNB()
# nb_model = GaussianNB()
nb_model.fit(train_x,train_y)

y_pred = nb_model.predict(test_x)

train_score = nb_model.score(train_x,train_y)
test_score = nb_model.score(test_x,test_y)
acc = accuracy_score(test_y,y_pred)
f1 = f1_score(test_y,y_pred,average='micro')

print("train_score:",train_score,"\n","test_score:",test_score)
print('acc:',acc,'\n','f1:',f1)

train_score: 0.4749956814648471 
 test_score: 0.437491903096256
acc: 0.437491903096256 
 f1: 0.437491903096256


## RF

In [47]:
from sklearn.ensemble import RandomForestClassifier

In [48]:
rf_model = RandomForestClassifier(n_estimators=500,max_depth=6,max_features=0.8,n_jobs=-1)
rf_model.fit(train_x,train_y)

y_pred = rf_model.predict(test_x)

train_score = rf_model.score(train_x,train_y)
test_score = rf_model.score(test_x,test_y)
acc = accuracy_score(test_y,y_pred)
f1 = f1_score(test_y,y_pred,average='micro')

print("train_score:",train_score,"\n","test_score:",test_score)
print('acc:',acc,'\n','f1:',f1)

train_score: 0.3493263085161513 
 test_score: 0.3316491773545796
acc: 0.3316491773545796 
 f1: 0.3316491773545796


## lgb

In [12]:
import lightgbm as lgb

### train

In [13]:
dtrain = lgb.Dataset(train_x,label=train_y) # free_raw_data=True
dtest = lgb.Dataset(test_x,label=test_y)

params = {
            'boosting':'gbdt',
#             'device_type':'gpu',
            'objective':'multiclass',
            'num_class':126,
#             'num_iterations':100,
            'metric': ['multi_error','multi_logloss'],
#             'min_gain_to_split':0. #'gamma'
            'max_depth':6,
            'lambda_l1':12, #'alpha'
            'lambda_l2':15, # 'lambda'
            'bagging_fraction':0.7,#'subsample'
            'feature_fraction':0.8,#'colsample_bytree'
#             'num_leaves':31,
            'learning_rate': 0.05, #eta
            'seed':666,
#             'silent':1,
            'missing':-1,
#             'early_stopping_round':
        }

lgb_model = lgb.train(params,dtrain,num_boost_round=1000,valid_sets=[dtrain,dtest],
                      early_stopping_rounds=500,#keep_training_booster = True,
                      verbose_eval=10)

# y_pred_prob = lgb_model.predict(dtest,ntree_limit=lgb_model.best_ntree_limit)
# y_pred = np.argmax(y_pred_prob,axis=1)

Training until validation scores don't improve for 500 rounds.
[10]	training's multi_logloss: 3.23533	training's multi_error: 0.757903	valid_1's multi_logloss: 3.42492	valid_1's multi_error: 0.7694
[20]	training's multi_logloss: 2.79187	training's multi_error: 0.613491	valid_1's multi_logloss: 3.08048	valid_1's multi_error: 0.657598
[30]	training's multi_logloss: 2.48629	training's multi_error: 0.548799	valid_1's multi_logloss: 2.85355	valid_1's multi_error: 0.610183
[40]	training's multi_logloss: 2.25671	training's multi_error: 0.504707	valid_1's multi_logloss: 2.68874	valid_1's multi_error: 0.581293
[50]	training's multi_logloss: 2.07667	training's multi_error: 0.468647	valid_1's multi_logloss: 2.5645	valid_1's multi_error: 0.560435
[60]	training's multi_logloss: 1.93204	training's multi_error: 0.43872	valid_1's multi_logloss: 2.46805	valid_1's multi_error: 0.54476
[70]	training's multi_logloss: 1.81325	training's multi_error: 0.413284	valid_1's multi_logloss: 2.39182	valid_1's multi

[600]	training's multi_logloss: 0.880197	training's multi_error: 0.190491	valid_1's multi_logloss: 1.98117	valid_1's multi_error: 0.479337
[610]	training's multi_logloss: 0.878414	training's multi_error: 0.189886	valid_1's multi_logloss: 1.98098	valid_1's multi_error: 0.478818
[620]	training's multi_logloss: 0.87683	training's multi_error: 0.189065	valid_1's multi_logloss: 1.9809	valid_1's multi_error: 0.478171
[630]	training's multi_logloss: 0.875181	training's multi_error: 0.18872	valid_1's multi_logloss: 1.98081	valid_1's multi_error: 0.478948
[640]	training's multi_logloss: 0.873656	training's multi_error: 0.187899	valid_1's multi_logloss: 1.98078	valid_1's multi_error: 0.479207
[650]	training's multi_logloss: 0.872063	training's multi_error: 0.187381	valid_1's multi_logloss: 1.98063	valid_1's multi_error: 0.479078
[660]	training's multi_logloss: 0.870603	training's multi_error: 0.187208	valid_1's multi_logloss: 1.98049	valid_1's multi_error: 0.479596
[670]	training's multi_logloss

In [31]:
lgb_model.save_model('./lgb_model.model')

### predict

In [49]:
test_data = app_desc_wv[train_num:]
y_pred_prob = lgb_model.predict(test_data,num_iteration=lgb_model.best_iteration)
y_pred = np.argmax(y_pred_prob,axis=1)

sub_df = train_test_data[train_num:]
sub_df['label'] = y_pred
sub_df.loc[:,'label'] = sub_df['label'].replace(id_dict)
sub_df.insert(2,'label1',sub_df['label'].str.slice(0,4))

sub_df = sub_df[['app_name','label1','label']]
sub_df.columns = ['id','label1','label2']
sub_df.to_csv('./lgb_predict.csv',index=False)

## xgb

In [27]:
import xgboost as xgb

### train

In [28]:
dtrain = xgb.DMatrix(train_x,label=train_y)
dtest = xgb.DMatrix(test_x,label=test_y)

params = {
            'booster':'gbtree',
            'tree_method':'gpu_hist',
            'objective':'multi:softprob',
            'num_class':126,
#             'scale_pos_weight': float(len(train_y)-sum(train_y))/float(sum(train_y)),
            'eval_metric': ['mlogloss','merror'],
            'gamma':0.,
            'max_depth':6,
            'alpha':15,
            'lambda':20,
            'subsample':0.7,
            'colsample_bytree':0.8,
#             'min_child_weight':6, 
            'eta': 0.1,
            'seed':666,
            'silent':1,
            'missing':-1,
        }
# xgb_cv = xgb.cv(params, dtrain, 
#            num_boost_round=1000, 
#            nfold=3, stratified=True,#folds=sfk
#            metrics='auc', maximize=True, 
#            early_stopping_rounds=500, 
#            as_pandas=True, verbose_eval=1, 
#            show_stdv=False, seed=0, shuffle=True)

xgb_model = xgb.train(params,dtrain,num_boost_round=1000,early_stopping_rounds=500,
                       evals=[(dtrain,'train'),(dtest,'valid')],verbose_eval=10)#xgb_model=xgb_trained_model)

[0]	train-mlogloss:4.36931	train-merror:0.687338	valid-mlogloss:4.39042	valid-merror:0.694131
Multiple eval metrics have been passed: 'valid-merror' will be used for early stopping.

Will train until valid-merror hasn't improved in 500 rounds.
[10]	train-mlogloss:3.06063	train-merror:0.564044	valid-mlogloss:3.25217	valid-merror:0.605907
[20]	train-mlogloss:2.54638	train-merror:0.517274	valid-mlogloss:2.84135	valid-merror:0.579091
[30]	train-mlogloss:2.24252	train-merror:0.48549	valid-mlogloss:2.61346	valid-merror:0.561083
[40]	train-mlogloss:2.03578	train-merror:0.456944	valid-mlogloss:2.4708	valid-merror:0.546573
[50]	train-mlogloss:1.8817	train-merror:0.431508	valid-mlogloss:2.37164	valid-merror:0.537116
[60]	train-mlogloss:1.76127	train-merror:0.411081	valid-mlogloss:2.29899	valid-merror:0.529732
[70]	train-mlogloss:1.663	train-merror:0.391043	valid-mlogloss:2.24428	valid-merror:0.522607
[80]	train-mlogloss:1.58111	train-merror:0.374935	valid-mlogloss:2.20083	valid-merror:0.519497
[

KeyboardInterrupt: 